In [ ]:
# script to compile reactions from a mechanism into RMG-database library form

In [1]:
import os

import rmgpy.chemkin
import rmgpy.data.kinetics
import rmgpy.data.thermo

from collections import OrderedDict


In [2]:
THERMO_LIB_NAME = 'my_thermo_lib'
KINETICS_LIB_NAME = 'my_kinetics_lib'

In [3]:
# load the RMG mechanism

base_dir = '/work/westgroup/harris.se/autoscience/fuels/butane/small_lib_20231110/'
chemkin = os.path.join(base_dir, 'chem_annotated.inp')
species_dict = os.path.join(base_dir, 'species_dictionary.txt')

species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin, dictionary_path=species_dict)



# Make thermo library

In [4]:
# compile it all into a single database and a single library which I'll call harris_butane
ark_thermo_database = rmgpy.data.thermo.ThermoDatabase()
ark_thermo_database.libraries[THERMO_LIB_NAME] = rmgpy.data.thermo.ThermoLibrary()
ark_thermo_database.libraries[THERMO_LIB_NAME].label = THERMO_LIB_NAME
ark_thermo_database.libraries[THERMO_LIB_NAME].entries = OrderedDict()
for i, species in enumerate(species_list):
    
    entry = rmgpy.data.base.Entry()
    entry.index = i
    entry.label = species.label
    entry.item = species.molecule[0]
    entry.data = species.thermo
    
    ark_thermo_database.libraries[THERMO_LIB_NAME].entries[entry.label] = entry
    

In [5]:
# save the results
ark_thermo_database.save_libraries(THERMO_LIB_NAME)

In [6]:
# try reading to test
# Load the new thermo library

thermo_lib = os.path.join(THERMO_LIB_NAME)
ark_thermo_database = rmgpy.data.thermo.ThermoDatabase()
ark_thermo_database.load_libraries(thermo_lib)

print(f'{len(ark_thermo_database.libraries[THERMO_LIB_NAME].entries)} entries loaded')


127 entries loaded


# Add kinetics (won't be able to read if duplicates)

In [7]:
# compile it all into a single database and a single library which I'll call harris_butane
ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
ark_kinetics_database.libraries[KINETICS_LIB_NAME] = rmgpy.data.kinetics.KineticsLibrary()
ark_kinetics_database.libraries[KINETICS_LIB_NAME].label = KINETICS_LIB_NAME
ark_kinetics_database.libraries[KINETICS_LIB_NAME].name = KINETICS_LIB_NAME
ark_kinetics_database.libraries[KINETICS_LIB_NAME].entries = OrderedDict()
for i, reaction in enumerate(reaction_list):

    entry = rmgpy.data.base.Entry()
    entry.index = i
    if not reaction.label:
        entry.label = reaction.to_labeled_str()
    else:
        entry.label = reaction.label
    entry.item = reaction
    entry.data = reaction.kinetics
    
    
    ark_kinetics_database.libraries[KINETICS_LIB_NAME].entries[entry.label] = entry

In [8]:
# save the results
ark_kinetics_database.save_libraries(KINETICS_LIB_NAME)

In [10]:
# read the results again
kinetics_lib = os.path.join(KINETICS_LIB_NAME)
ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
ark_kinetics_database.load_libraries(kinetics_lib)

print(f'{len(ark_kinetics_database.libraries[KINETICS_LIB_NAME].entries)} entries loaded')


ERROR:root:Problem loading reaction library 'my_kinetics_lib/my_kinetics_lib/reactions.py'


DatabaseError: Unexpected duplicate reaction CH2CHO + CH2CHO <=> CH2CO + CH3CHO in kinetics library my_kinetics_lib. Reaction index 542 matches index 393.